# Bayesian SEM

## Preliminaries

In [ ]:
load("first_test.RData")

In [4]:
library(manymome)
library(lavaan)
library(brms)

Loading required package: Rcpp

Loading 'brms' package (version 2.18.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar




In [2]:
dat <- data_sem 
head(dat)

,x01,x02,x03,x04,x05,x06,x07,x08,x09,x10,x11,x12,x13,x14
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.23576013,-1.05037417,-0.2040220,2.7002823,-0.11505670,2.1698611,1.6852091,0.2360732,-1.9410730,-2.1475869,-0.8945538,-0.51234386,-0.6378251,-1.5415323
2,0.11337970,0.27366418,0.8584178,-0.4686163,-1.03172258,0.7874647,-0.2813761,-0.9216282,0.7760754,-1.4925370,-0.7365507,1.03750725,-0.7504301,-1.4359552
3,-0.08915241,-0.05083648,2.4274752,-0.2252800,-0.06686246,-1.6325857,-1.8842986,-2.4352690,-0.3012754,-0.2454858,0.4550789,-0.68057390,0.1880389,-0.6560311
4,-0.48256707,1.14124531,-0.7289318,-0.5325998,1.02918776,-0.5794417,-0.9199805,-1.5210671,-1.5298175,-0.5369419,-2.1625877,-1.57768509,-0.8356664,-0.1963328
5,-0.82143162,-0.32768509,-0.8794389,-0.2084861,0.30020723,-2.0980021,-0.4602843,-1.7961560,-0.1309144,-2.7187970,-0.4373113,-1.16013401,0.2917883,-2.5906322
6,0.90501420,0.32949507,0.2525691,0.9569735,-1.84036094,-0.3608671,0.8296289,-0.4051225,-1.0937565,-2.0898469,0.3949706,-0.01419469,-0.3025810,2.2394808


## lavaan syntax fails

In [8]:
# Writing the formulas that describe the dependency structure of the model
# This fails

f1_form <- bf(f1 =~ x01 + x02 + x03)
f2_form <- bf(f2 =~ x04 + x05 + x06 + x07)
f3_form <- bf(f3 =~ x08 + x09 + x10)
f4_form <- bf(f4 =~ x11 + x12 + x13 + x14)
f3_lat <- bf(f3 ~ f1 + f2)
f4_lat <- bf(f4 ~ f1 + f3)

fit_brms <- brm(f1_form + f2_form + f3_form + f4_form + f3_lat + f4_lat + set_rescor(FALSE), data = dat, iter = 2000, chains = 2, cores = 4, seed = 123)

ERROR: Error in brmsformula(formula, ..., flist = flist, family = family, autocor = autocor, : argument "formula" is missing, with no default


## Alternatives

Apparently, brms does not have this capability natively. There was discussion [here](https://github.com/paul-buerkner/brms/issues/304), and it looks like it will be implemented in brms 3.0 per [this](https://github.com/paul-buerkner/brms/milestone/20). But nothing yet. 

Currently, there seem to be two alternatives: 
- Mimic this behavior using the `mi()` function from brms, as described [here](https://scottclaessens.github.io/blog/2020/brmsLV/)
- Use blavaan instead. It was originally built on top of JAGS, but now it works with Stan. See [this](https://arxiv.org/pdf/2008.07733.pdf)


In [13]:
# For reference, let us fit a simple model with lavaan
mod <- 
"
f2 =~ x01 + x02
x03 ~ f2
"

fit_lavaan <- sem(mod, data = dat)
parameterEstimates(fit_lavaan)

lhs,op,rhs,est,se,z,pvalue,ci.lower,ci.upper
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
f2,=~,x01,1.0000000,0.0000000,NA,NA,1.0000000,1.000000
f2,=~,x02,0.7968281,0.1340520,5.944170,2.778608e-09,0.5340909,1.059565
x03,~,f2,1.3631511,0.2402049,5.674951,1.387281e-08,0.8923581,1.833944
x01,~~,x01,1.0495644,0.1625866,6.455418,1.079208e-10,0.7309005,1.368228
x02,~~,x02,1.0840369,0.1340419,8.087299,6.661338e-16,0.8213196,1.346754
x03,~~,x03,0.8706706,0.2466138,3.530502,4.147723e-04,0.3873163,1.354025
f2,~~,f2,0.7797276,0.1945913,4.007001,6.149460e-05,0.3983356,1.161120


In [15]:
# Using mi()

# Creating empty column for the mediator
dat$f1 <- as.numeric(NA)

# Defining the dependency of the mediator on observed variables: 
bf1 <- bf(x01 ~ 0 + mi(f1))
bf2 <- bf(x02 ~ 0 + mi(f1))

# Telling brms that the mediator is a latent variable
bf3 <- bf(f1 | mi() ~ 0)

# Defining the dependency of the outcome on the mediator
bf4 <- bf(x03 ~ 0 + mi(f1))

# Fitting the brms object 
fit_brms <- brm(bf1 + bf2 + bf3 + bf4 + set_rescor(FALSE), data = dat, iter = 2000, chains = 2, cores = 4, seed = 123, prior = prior(normal(1, 0.00001), coef = mif1, resp = x01))

summary(fit_brms)

Compiling Stan program...

Start sampling

Warning message:
“There were 1836 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.08, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may b

 Family: MV(gaussian, gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity
         mu = identity; sigma = identity 
Formula: x01 ~ 0 + mi(f1) 
         x02 ~ 0 + mi(f1) 
         f1 | mi() ~ 0 
         x03 ~ 0 + mi(f1) 
   Data: dat (Number of observations: 200) 
  Draws: 2 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 2000

Population-Level Effects: 
         Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
x01_mif1     1.00      0.00     1.00     1.00 1.00     2144     1381
x02_mif1     0.83      0.14     0.58     1.13 1.01      202      604
x03_mif1     1.47      0.31     0.99     2.24 1.08       27       27

Family Specific Parameters: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma_x01     1.05      0.08     0.89     1.20 1.04       61      373
sigma_x02     1.05      0.07     0.93     1.20 1.01 

In [1]:
save(file = "first_test.RData")

Warning message in save(file = "first_test.RData"):
“nothing specified to be save()d”
